In [2]:
import numpy as np
import pandas as pd
import scipy
import json
from collections import Counter
import copy
import random
import sklearn

In [82]:
%%time
# 입력데이터 로드
train = pd.read_json('train.json', typ = 'frame')
genre = pd.read_json('genre_gn_all.json', typ = 'series')
meta = pd.read_json('song_meta.json', typ = 'frame')

CPU times: total: 15.5 s
Wall time: 18.2 s


In [7]:
train.head()

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000


In [16]:
# 폴킴, 멜로망스 노래 songid를 list로 만든다.

new_meta = meta[['id','artist_name_basket','song_gn_gnr_basket','song_name']]
new_meta["artist_name_basket"] = new_meta["artist_name_basket"].apply(lambda x: x[0] if x else x)

paulkim = new_meta[new_meta["artist_name_basket"] == "폴킴"]
melomance = new_meta[new_meta["artist_name_basket"] == "멜로망스"]

total = pd.concat([paulkim, melomance]).reset_index().drop(columns=["index"])

# 폴킴, 멜로망스 노래중 각 5개씩 뽑는다.
playlists = []
for _ in range(2):
    pk = random.sample(paulkim["id"].tolist(),5)
    mm = random.sample(melomance["id"].tolist(),5)
    songlist = pk + mm
    random.shuffle(songlist)
    playlists.append(songlist)

for playlist in playlists:
    songs = total[total["id"].apply(lambda x: x in playlist)]["song_name"]
    print(songs.tolist())

['비', '꽃비', '초록빛', 'Not Over Yet', '이런 엔딩', '작은 행복', 'Moonlight', '영화관에서', '자장가', '축제']
['만나볼래요', '커피한잔할래요', '허전해', '비', 'Not Over Yet', '네 옆에 있을게', 'Moonlight', 'You', '나나나', '네 옆에 있을게 (Inst.)']


C:\Users\User\AppData\Local\Temp\ipykernel_9144\3515485258.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_meta["artist_name_basket"] = new_meta["artist_name_basket"].apply(lambda x: x[0] if x else x)


In [27]:
paulkim

,id,artist_name_basket,song_gn_gnr_basket,song_name
13519,13519,폴킴,[GN0400],좋은 사람
17619,17619,폴킴,[GN0400],Game
24913,24913,폴킴,[GN0400],만나볼래요
36413,36413,폴킴,[GN0400],널 듣고만 있어
79813,79813,폴킴,[GN0400],Christmas Love (Inst.)
...,...,...,...,...
670390,670390,폴킴,"[GN1500, GN0100]",사랑 알 수 없나봐
670510,670510,폴킴,[GN0400],Wanna Love You
675393,675393,폴킴,[GN0400],Her
691847,691847,폴킴,[GN0400],편지


In [8]:
# My Own Playlist json file 만들기

playlists_json = [{"songs": playlists[0]},{"songs": playlists[1]}]
with open("ownplaylist.json", 'w') as json_file:
    json.dump(playlists_json, json_file)


In [11]:
# Random Select

!python ownrandom.py run \
 	--train_fname=arena_data/orig/train.json \
 	--question_fname=arena_data/questions/val.json

Loading train file...
Loading question file...
Writing answers...



100%|██████████| 23015/23015 [00:00<00:00, 77600.99it/s]


In [17]:
# Genre별 Most popular song 찾는 code (From Week4.ipynb)

k = 10 # number of popular song

genre_df = genre.to_frame().reset_index()
genre_df = genre_df.rename(columns = {"index":"genre_code",0:"genre_name"})

genre_code_name_dict = dict()
for _, row in genre_df.iterrows():
    genre_code_name_dict[row["genre_code"]] = row["genre_name"]

#genre_code_name_dict

def mapping_function(data, col1, col2):
    # 플레이리스트 아이디(col1)와 수록곡(col2) 추출
    plylst_song_map = data[[col1, col2]]

    # unnest col2
    plylst_song_map_unnest = np.dstack(
        (
            np.repeat(plylst_song_map[col1].values, list(map(len, plylst_song_map[col2]))),
            np.concatenate(plylst_song_map[col2].values)
        )
    )

    # unnested 데이터프레임 생성 : plylst_song_map
    plylst_song_map = pd.DataFrame(data = plylst_song_map_unnest[0], columns = plylst_song_map.columns)
    plylst_song_map[col1] = plylst_song_map[col1].astype(str)
    plylst_song_map[col2] = plylst_song_map[col2].astype(str)

    # unnest 객체 제거
    del plylst_song_map_unnest
    return plylst_song_map

playlst_song_map = mapping_function(train, 'id', 'songs')
agg = pd.DataFrame(playlst_song_map['songs'].value_counts()).reset_index()
agg.columns = ['곡', '플레이리스트내의 등장횟수']
meta['id'] = meta['id'].astype(str)
agg = agg.merge(meta[['id', 'song_name', 'song_gn_gnr_basket']], how='left', left_on='곡', right_on='id')
agg.drop(['곡','id'], axis=1, inplace=True)
agg = agg.rename(columns={'song_gn_gnr_basket':'genre_code'})
agg = agg[agg["genre_code"].apply(lambda x: len(x) > 0)]

# 총 장르 수 254, 대분류 장르 30, 소분류 장르 224

gnr_code_list = list(genre.index) # 총 장르 list

gnr_code_name = pd.DataFrame(genre, columns = ['gnr_name']).reset_index().rename(columns = {'index' : 'gnr_code'})
large_gnr_code_list = gnr_code_name[gnr_code_name['gnr_code'].str[-2:] == '00'] # 대분류 장르 list

c = Counter()
for gnr_code in meta["song_gn_gnr_basket"]:
    if len(gnr_code) == 0: continue
    for sub_item in gnr_code:
        if sub_item in gnr_code_list:
            c.update([sub_item])

gnrname_number = pd.DataFrame.from_dict(c, orient='index').reset_index().rename(columns = {0:'number','index':'genre_name'})
gnrcode_number = pd.DataFrame.from_dict(c, orient='index').reset_index().rename(columns = {0:'number','index':'genre_code'})
gnrname_number["genre_name"] = gnrname_number["genre_name"].apply(lambda code: genre[code])
gnrcode_number = gnrcode_number.sort_values(by=["number"], ascending=[False]).reset_index(drop=True)
gnrname_number = gnrname_number.sort_values(by=["number"], ascending=[False]).reset_index(drop=True)
gnrname_number.index += 1
#gnrname_number.head(10) # 장르별 곡 수 (장르 랭킹)
#gnrcode_number.head(10)
#gnrname_10

all_genre_code = gnrcode_number["genre_code"][:].tolist()

genre_song_dict = dict()
for genre_code in all_genre_code:
    genre_song_dict.setdefault(genre_code, [])

tot = 0
for i, row in agg.iterrows():
    genre_code_list = row["genre_code"]
    song_name = row["song_name"]
    for g in genre_code_list:
        if not g in all_genre_code: continue
        if len(genre_song_dict[g]) == k: continue
        genre_song_dict[g].append(song_name)
        tot += 1
    if tot == len(all_genre_code)*k: break

#genre_song_dict

genre_top10_song = pd.DataFrame(genre_song_dict)
genre_top10_song.index = genre_top10_song.index+1
genre_top10_song = genre_top10_song.T.reset_index()
genre_top10_song = genre_top10_song.rename(columns = {'index':'genre_code'})
genre_top10_song["genre_code"] = genre_top10_song["genre_code"].apply(lambda code: genre_code_name_dict[code])
genre_top10_song.index = genre_top10_song.index+1
genre_top10_song = genre_top10_song.rename(columns = {'genre_code': 'genre_name'})
#genre_top10_song

merged_df = genre_top10_song.merge(gnrname_number, on='genre_name', how='left')

desired_columns = ['genre_name', 'number']+[i for i in range(1,k+1)]
merged_df = merged_df[desired_columns]
merged_df.index = merged_df.index+1
merged_df.head(5)

,genre_name,number,1,2,3,4,5,6,7,8,9,10
1,POP,77101,Lost Stars,I`m Not The Only One,Shape of You,YOUTH,Sugar,Uptown Funk (Feat. Bruno Mars),Love Yourself,Happy (Gru`s Theme From Despicable Me 2),One Call Away,Sunday Morning
2,록/메탈,63926,Lost Stars,Thinking Out Loud,Sugar,Fix You,Counting Stars,The Scientist,Photograph,Lucky (Feat. Colbie Caillat),Lucky Strike,Everglow
3,록/메탈,29690,Lost Stars,Thinking Out Loud,Sugar,Fix You,Counting Stars,The Scientist,Photograph,Lucky (Feat. Colbie Caillat),Lucky Strike,Everglow
4,일렉트로니카,60144,Closer (Feat. Halsey),Something Just Like This,The Ocean (Feat. Shy Martin) (Radio Edit.),This Is What You Came For (Feat. Rihanna),Faded,How To Love (Feat. Sofia Reyes),"Feels (Feat. Pharrell Williams, Katy Perry & B...",Get Lucky (Feat. Pharrell Williams),Bad (Feat. Vassy) (Radio Edit),Slide (Feat. Frank Ocean & Migos)
5,일렉트로니카,6400,Closer (Feat. Halsey),Something Just Like This,The Ocean (Feat. Shy Martin) (Radio Edit.),This Is What You Came For (Feat. Rihanna),Faded,How To Love (Feat. Sofia Reyes),"Feels (Feat. Pharrell Williams, Katy Perry & B...",Get Lucky (Feat. Pharrell Williams),Bad (Feat. Vassy) (Radio Edit),Slide (Feat. Frank Ocean & Migos)


In [4]:
# Playlist의 most popular genre 찾기

genre_code_counter = Counter()
for playlist in playlists:
    for songid in playlist:
        genre_list = total[total["id"]==songid]["song_gn_gnr_basket"]
        for genre_code in genre_list:
            genre_code_counter.update(genre_code)

most_popular_genre = genre[genre_code_counter.most_common(1)[0][0]]

# 해당 genre에서 가장 popular한 곡들을 추천

merged_df[merged_df["genre_name"] == most_popular_genre].reset_index().drop(columns=['number','genre_name','index'])

NameError: name 'playlists' is not defined

In [130]:
# binary playlist by track matrix 만들기
sub_train = train.loc[:len(train)//10]
sub_songs = meta.loc[:len(meta)//10]
num_playlist = len(sub_train) # 11508
num_songs = len(sub_songs) # 70799

playlist_by_track = np.zeros((num_playlist,num_songs), dtype=int)

for i, row in sub_train.iterrows():
    #playlist_id = row["id"]
    songs = row["songs"]
    for songid in songs:
        if songid < num_songs:
            playlist_by_track[i][songid] = 1

In [131]:
print(num_playlist)
print(num_songs)

11508
70799


In [132]:
playlist_by_track.sort(axis=1)
playlist_by_track = -np.sort(-playlist_by_track)

In [133]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=128)
tsvd = svd.fit_transform(playlist_by_track)

reconstructed_matrix = np.dot(tsvd, svd.components_)
reconstructed_matrix = np.sort(reconstructed_matrix, axis=1)[:, ::-1]

In [128]:
np.set_printoptions(linewidth=np.inf)

k = 1
print("approx:",reconstructed_matrix[k][:10])
print("binary:",playlist_by_track[k][:10])

approx: [1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.06177349e-15 7.62327252e-16 5.45356167e-16 5.03030714e-16 4.60166002e-16]
binary: [1 1 1 1 1 0 0 0 0 0]


In [129]:
num_rows = playlist_by_track.shape[0]
next_values = []
for i in range(num_rows):
    count = 0
    while True:
        if playlist_by_track[i][count] == 0:
            break
        count += 1
    next_values.append(reconstructed_matrix[i][count])

print(max(next_values))

1.234749225856983e-15


In [33]:
print(meta[meta["id"]==13519]["song_name"])

for songid in similar_songs:
    print(meta[meta["id"]==songid]["song_name"])

13519    좋은 사람
Name: song_name, dtype: object
23601    Did You Hear The Rain?
Name: song_name, dtype: object
23595    그래 (Say) (Inst.)
Name: song_name, dtype: object
23596    Title Music From Satyajit Ray&#39;s film JALSA...
Name: song_name, dtype: object
23597    Telemann : Suite In A Minor TWV.55/A2 - IV. Re...
Name: song_name, dtype: object
23598    Merry Christmas Baby
Name: song_name, dtype: object
